In [1]:
import os

In [2]:
%pwd

'c:\\Users\\16676\\NLP-Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\16676\\NLP-Text-Summarizer'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    data_path:Path
    tokenizer_path:Path


In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories


In [57]:
class ConfigurationManager:

    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        
                 

        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation
        create_directories([config.root_dir])
       

        data_transformation_config= DataTransformationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config
    



In [ ]:
import os
from src.textSummarizer.logging import logger

In [51]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk


In [61]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)


    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=1024,truncation=True)
        return{
            'input_ids':input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
            
   
        }    
    
    def convert(self):
        dataset_samsum=load_from_disk(self.config.dataset_path)
        dataset_samsum=dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

      

In [63]:
try:
        config=ConfigurationManager()
        data_transformation_config=config.get_data_transformation_config()
        data_transformation=DataTransformation(config=data_transformation_config)
        data_transformation.convert()
except Exception as e:
        raise e
        

[2023-07-16 22:23:29,337:INFO: common:yaml file: config\config.yaml loaded successfully.]
[2023-07-16 22:23:29,343:INFO: common:yaml file: params.yaml loaded successfully.]
[2023-07-16 22:23:29,347:INFO: common:created directory at: artifacts]
[2023-07-16 22:23:29,352:INFO: common:created directory at: artifacts/data_transformation]


TypeError: DataTransformationConfig.__init__() got an unexpected keyword argument 'root_dir'